<a href="https://colab.research.google.com/github/hanee0j/NLP/blob/main/%5BNLP%5D_Lab_3_1_Examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

seed = 1337

torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

### Example 1

In [2]:
class MultilayerPerceptron(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        """
        Args:
            input_dim (int): the size of the input vectors
            hidden_dim (int): the output size of the first Linear layer
            output_dim (int): the output size of the second Linear layer
        """
        super(MultilayerPerceptron, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x_in, apply_softmax=False):
        """The forward pass of the MLP
        
        Args:
            x_in (torch.Tensor): an input data tensor. 
                x_in.shape should be (batch, input_dim)
            apply_softmax (bool): a flag for the softmax activation
                should be false if used with the Cross Entropy losses
        Returns:
            the resulting tensor. tensor.shape should be (batch, output_dim)
        """
        intermediate = F.relu(self.fc1(x_in))
        output = self.fc2(intermediate)
        
        if apply_softmax:
            output = F.softmax(output, dim=1)
        return output

### Example 2

In [3]:
batch_size = 2 # number of samples input at once
input_dim = 3
hidden_dim = 100
output_dim = 4

# Initialize model
mlp = MultilayerPerceptron(input_dim, hidden_dim, output_dim)
print(mlp)

MultilayerPerceptron(
  (fc1): Linear(in_features=3, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=4, bias=True)
)


### Example 3

In [4]:
def describe(x):
    print("Type: {}".format(x.type()))
    print("Shape/size: {}".format(x.shape))
    print("Values: \n{}".format(x))

In [5]:
# Inputs
x_input = torch.rand(batch_size, input_dim)
describe(x_input)

Type: torch.FloatTensor
Shape/size: torch.Size([2, 3])
Values: 
tensor([[0.8329, 0.4277, 0.4363],
        [0.9686, 0.6316, 0.8494]])


In [6]:
y_output = mlp(x_input, apply_softmax=False)
describe(y_output)

Type: torch.FloatTensor
Shape/size: torch.Size([2, 4])
Values: 
tensor([[-0.2456,  0.0723,  0.1589, -0.3294],
        [-0.3497,  0.0828,  0.3391, -0.4271]], grad_fn=<AddmmBackward0>)


### Example 4

In [7]:
y_output = mlp(x_input, apply_softmax=True)
describe(y_output)

Type: torch.FloatTensor
Shape/size: torch.Size([2, 4])
Values: 
tensor([[0.2087, 0.2868, 0.3127, 0.1919],
        [0.1832, 0.2824, 0.3649, 0.1696]], grad_fn=<SoftmaxBackward0>)


### Example 5

In [8]:
class MultilayerPerceptron(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        """
        Args:
            input_dim (int): the size of the input vectors
            hidden_dim (int): the output size of the first Linear layer
            output_dim (int): the output size of the second Linear layer
        """
        super(MultilayerPerceptron, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x_in, apply_softmax=False):
        """The forward pass of the MLP
        
        Args:
            x_in (torch.Tensor): an input data tensor. 
                x_in.shape should be (batch, input_dim)
            apply_softmax (bool): a flag for the softmax activation
                should be false if used with the Cross Entropy losses
        Returns:
            the resulting tensor. tensor.shape should be (batch, output_dim)
        """
        intermediate = F.relu(self.fc1(x_in))
        output = self.fc2(F.dropout(intermediate, p=0.5))
        
        if apply_softmax:
            output = F.softmax(output, dim=1)
        return output

batch_size = 2 # number of samples input at once
input_dim = 3
hidden_dim = 100
output_dim = 4

# Initialize model
mlp = MultilayerPerceptron(input_dim, hidden_dim, output_dim)
print(mlp)

y_output = mlp(x_input, apply_softmax=False)
describe(y_output)

MultilayerPerceptron(
  (fc1): Linear(in_features=3, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=4, bias=True)
)
Type: torch.FloatTensor
Shape/size: torch.Size([2, 4])
Values: 
tensor([[ 0.0193,  0.0275,  0.2319,  0.3032],
        [-0.5323,  0.3183,  0.4194, -0.0205]], grad_fn=<AddmmBackward0>)


### Example 6

In [9]:
batch_size = 2
one_hot_size = 10
sequence_width = 7
data = torch.randn(batch_size, one_hot_size, sequence_width)
conv1 = nn.Conv1d(in_channels=one_hot_size, out_channels=16, kernel_size=3)
intermediate1 = conv1(data)
print(data.size())
print(intermediate1.size())

torch.Size([2, 10, 7])
torch.Size([2, 16, 5])


### Example 7

In [10]:
conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3)
conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3)

intermediate2 = conv2(intermediate1)
intermediate3 = conv3(intermediate2)

print(intermediate2.size())
print(intermediate3.size())

torch.Size([2, 32, 3])
torch.Size([2, 64, 1])


In [11]:
y_output = intermediate3.squeeze()
print(y_output.size())

torch.Size([2, 64])


In [12]:
intermediate2.mean(dim=0).mean(dim=1).sum()

tensor(-0.0493, grad_fn=<SumBackward0>)

### Example 8

In [13]:
# Method 2 of reducing to feature vectors
print(intermediate1.view(batch_size, -1).size())

# Method 3 of reducing to feature vectors
print(torch.mean(intermediate1, dim=2).size())
# print(torch.max(intermediate1, dim=2).size())
# print(torch.sum(intermediate1, dim=2).size())

torch.Size([2, 80])
torch.Size([2, 16])


### Example 9
The full model will not be reproduced here. Instead, we will just show batch norm being used.  

In [14]:
conv1 = nn.Conv1d(in_channels=one_hot_size, out_channels=16, kernel_size=3)
conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3)
conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3)

conv1_bn = nn.BatchNorm1d(num_features=16)
conv2_bn = nn.BatchNorm1d(num_features=32)
    
intermediate1 = conv1_bn(F.relu(conv1(data)))
intermediate2 = conv2_bn(F.relu(conv2(intermediate1)))
intermediate3 = conv3(intermediate2)

print(intermediate1.size())
print(intermediate2.size())
print(intermediate3.size())

torch.Size([2, 16, 5])
torch.Size([2, 32, 3])
torch.Size([2, 64, 1])


Note: BatchNorm computes its statistics over the batch and sequence dimensions. In other words, the input to each batchnorm1d is a tensor of size `(B, C, L)` (where b=batch, c=channels, and l=length). Each `(B, L)` slice should have 0-mean.  This reduces covariate shift. 

In [15]:
intermediate2.mean(dim=(0, 2))

tensor([ 0.0000e+00,  2.9802e-08, -2.9802e-08, -3.9736e-08,  9.9341e-09,
        -1.9868e-08,  4.9671e-08,  5.9605e-08,  2.9802e-08,  0.0000e+00,
         3.9736e-08,  2.9802e-08,  0.0000e+00,  2.9802e-08, -9.9341e-09,
         0.0000e+00,  3.9736e-08,  7.9473e-08,  1.9868e-08,  1.9868e-08,
        -2.9802e-08, -3.9736e-08, -2.1110e-08,  3.9736e-08,  3.9736e-08,
         1.9868e-08,  0.0000e+00,  0.0000e+00,  0.0000e+00, -3.9736e-08,
        -1.9868e-08,  4.4703e-08], grad_fn=<MeanBackward1>)


## Bonus Examples

In chapter 4, we cover convolutions. Below are code examples which instantiate the convolutions with various hyper parameter settings. 

In [16]:
x = torch.randn(1, 2, 3, 3)
describe(x)

conv1 = nn.Conv2d(in_channels=2, out_channels=1, kernel_size=2)
describe(conv1.weight)
describe(conv1(x))

Type: torch.FloatTensor
Shape/size: torch.Size([1, 2, 3, 3])
Values: 
tensor([[[[-1.3034,  0.9247,  1.2295],
          [-1.1446,  0.5722, -0.1904],
          [ 0.2819,  1.6849, -0.4500]],

         [[-1.5986,  0.8513,  0.3467],
          [ 0.4521, -0.8266, -0.5102],
          [ 0.1659, -0.0821,  0.1389]]]])
Type: torch.FloatTensor
Shape/size: torch.Size([1, 2, 2, 2])
Values: 
Parameter containing:
tensor([[[[ 0.2706,  0.3010],
          [-0.3514,  0.1179]],

         [[-0.3259,  0.1782],
          [ 0.0018,  0.2244]]]], requires_grad=True)
Type: torch.FloatTensor
Shape/size: torch.Size([1, 1, 2, 2])
Values: 
tensor([[[[ 0.7289, -0.0895],
          [-0.5052, -0.4926]]]], grad_fn=<ConvolutionBackward0>)


In [17]:
x = torch.randn(1, 1, 3, 3)
describe(x)

conv1 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=2)
describe(conv1.weight)
describe(conv1(x))

Type: torch.FloatTensor
Shape/size: torch.Size([1, 1, 3, 3])
Values: 
tensor([[[[-1.4027, -0.4858, -1.6482],
          [-3.2619,  0.1149,  0.7911],
          [-0.0382, -0.1485, -0.6093]]]])
Type: torch.FloatTensor
Shape/size: torch.Size([2, 1, 2, 2])
Values: 
Parameter containing:
tensor([[[[ 0.0806,  0.4423],
          [-0.1840, -0.4620]]],


        [[[-0.1611,  0.1878],
          [ 0.3422, -0.0861]]]], requires_grad=True)
Type: torch.FloatTensor
Shape/size: torch.Size([1, 2, 2, 2])
Values: 
tensor([[[[ 3.5492e-01, -1.0193e+00],
          [-8.9103e-04,  8.0362e-01]],

         [[-5.2371e-01,  2.0747e-01],
          [ 1.0143e+00,  5.9923e-01]]]], grad_fn=<ConvolutionBackward0>)
